# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Lab 13** </center>

---

**Fecha**: 09 Mayo 2025

**Nombre del Equipo**: Arriba Linux

**Integrantes del Equipo**: Tirzah Peniche Barba / Ana Cristina Luna Arellano / Juan Pedro Bihouet

**Profesor**: Dr. Pablo Camarillo Ramirez

In [2]:
import findspark
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MLSpark-K-means-ArribaLinux") \
    .master("spark://d9c3cc2bade8:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext
spark.conf.set("spark.sql.shuffle.partitions", "5")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/09 06:24:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
from ArribaLinux.spark_utils import SparkUtils

def generate_schema(columns):
    return [(col, "float") for col in columns]

# Load the wine dataset and apply schema

In [7]:
columns = [
    "Alcohol", "Malic_Acid", "Ash", "Ash_Alcanity", "Magnesium",
    "Total_Phenols", "Flavanoids", "Nonflavanoid_Phenols", "Proanthocyanins",
    "Color_Intensity", "Hue", "OD280", "Proline"
]

schema = SparkUtils.generate_schema(generate_schema(columns))

wine_df = spark.read.schema(schema).option("header", "true").csv("/home/jovyan/notebooks/data/wine-clustering.csv")
wine_df.show(30)


+-------+----------+----+------------+---------+-------------+----------+--------------------+---------------+---------------+----+-----+-------+
|Alcohol|Malic_Acid| Ash|Ash_Alcanity|Magnesium|Total_Phenols|Flavanoids|Nonflavanoid_Phenols|Proanthocyanins|Color_Intensity| Hue|OD280|Proline|
+-------+----------+----+------------+---------+-------------+----------+--------------------+---------------+---------------+----+-----+-------+
|  14.23|      1.71|2.43|        15.6|    127.0|          2.8|      3.06|                0.28|           2.29|           5.64|1.04| 3.92| 1065.0|
|   13.2|      1.78|2.14|        11.2|    100.0|         2.65|      2.76|                0.26|           1.28|           4.38|1.05|  3.4| 1050.0|
|  13.16|      2.36|2.67|        18.6|    101.0|          2.8|      3.24|                 0.3|           2.81|           5.68|1.03| 3.17| 1185.0|
|  14.37|      1.95| 2.5|        16.8|    113.0|         3.85|      3.49|                0.24|           2.18|            7.

# Assemble features into a single vector column for K-Means

In [9]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=columns, outputCol="features")
assembled_df = assembler.transform(wine_df)

# Train and evaluate K-Means 

In [10]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

k_values = [2, 10, 15, 20]
models = []
predictions = []
silhouette_scores = []

for k in k_values:
    kmeans = KMeans().setK(k).setSeed(19)
    model = kmeans.fit(assembled_df)
    models.append(model)
    prediction = model.transform(assembled_df)
    predictions.append(prediction)

    evaluator = ClusteringEvaluator()
    silhouette = evaluator.evaluate(prediction)
    silhouette_scores.append(silhouette)

    print(f"Silhouette score for k = {k}: {silhouette}")


25/05/09 06:37:51 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/05/09 06:37:51 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


Silhouette score for k = 2: 0.821360351333219


Silhouette score for k = 10: 0.6099911274077665


Silhouette score for k = 15: 0.6663309347635443


Silhouette score for k = 20: 0.6341822740902026


# Cluster centers for each trained model

In [11]:
for i, model in enumerate(models):
    print(f"Cluster Centers for k = {k_values[i]}:")
    for center in model.clusterCenters():
        print(center)
    print()


Cluster Centers for k = 2:
[1.27028455e+01 2.54455285e+00 2.33910569e+00 2.04081301e+01
 9.68130081e+01 2.06211382e+00 1.64146342e+00 3.92682924e-01
 1.45406503e+00 4.85138211e+00 9.08617886e-01 2.40821138e+00
 5.65869919e+02]
[1.36665455e+01 1.87072727e+00 2.42781818e+00 1.74527272e+01
 1.06290909e+02 2.81618182e+00 2.89654548e+00 2.92909090e-01
 1.89690911e+00 5.52036361e+00 1.06665455e+00 3.06672727e+00
 1.15172727e+03]

Cluster Centers for k = 10:
[1.39289474e+01 1.78157895e+00 2.48684212e+00 1.70789474e+01
 1.05315789e+02 2.90578946e+00 3.09947371e+00 2.84736841e-01
 1.93789475e+00 6.37105263e+00 1.11000000e+00 3.00894736e+00
 1.36747368e+03]
[1.26437931e+01 2.80896552e+00 2.26724137e+00 2.07241379e+01
 9.27586207e+01 1.80034481e+00 1.35379312e+00 4.06206897e-01
 1.34310344e+00 4.53482758e+00 8.73793103e-01 2.26344827e+00
 5.29827586e+02]
[1.32000002e+01 2.87375000e+00 2.45999998e+00 1.94875001e+01
 1.13375000e+02 2.14125003e+00 1.78874999e+00 3.67499998e-01
 1.60374998e+00 6.2550

In [12]:
sc.stop()